In [3]:
import json
import requests


import requests

url = "https://api.openaq.org/v2/sources?limit=100&page=1&offset=0&sort=asc&order_by=sourceName"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

json_data = response.json()
json_data['results'][0]


{'data': {'url': 'https://www.adairquality.ae/',
  'data_avg_dur': None,
  'organization': None,
  'lifecycle_stage': None},
 'readme': None,
 'sourceId': 9800851,
 'locations': 81,
 'sourceName': 'Abu Dhabi Air Quality',
 'sourceSlug': None}

In [4]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column
from sqlalchemy.sql.sqltypes import Integer, String

Base = declarative_base()

class DBSources(Base):
    __tablename__="source"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(250), nullable=False)
    sourceId = Column(Integer)
    location = Column(Integer)